<a href="https://colab.research.google.com/github/GuidoLorenzetti/TP2-AA2-Lorenzetti_Farias_Garcia/blob/main/Ejercicio%202/EJ2_Test_TP2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba

Utilizamos google colab para esta parte por incompatibilidades con las librerías del entorno de entrenamiento.

In [1]:
from google.colab import drive
# Mount your Google Drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import tensorflow as tf

# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    model_chars = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_char.h5')


In [3]:
with tf.keras.utils.custom_object_scope({'loss': loss}):
    model_words = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_words.h5')


# Análisis de parámetros

Revisamos como se componen los modelos previamente entrenados.

In [4]:
model_chars.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           20992     
                                                                 
 gru_2 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_2 (Dense)             (64, None, 82)            84050     
                                                                 
Total params: 4043346 (15.42 MB)
Trainable params: 4043346 (15.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model_words.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (64, None, 256)           15559168  
                                                                 
 gru_3 (GRU)                 (64, None, 1024)          3938304   
                                                                 
 dense_3 (Dense)             (64, None, 60778)         62297450  
                                                                 
Total params: 81794922 (312.02 MB)
Trainable params: 81794922 (312.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## From train

In [6]:
import tensorflow_datasets as tfds
import numpy as np
import os

# Cargar el dataset de noticias
dataset, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# Convertir a texto plano
train_texts = []
for text, label in tfds.as_numpy(train_dataset):
    train_texts.append(text.decode('utf-8'))

# Concatenar todos los textos en uno solo
text = ' '.join(train_texts)
print(f'Texto total: {len(text)} caracteres')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.M4MEXE_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.M4MEXE_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.
Texto total: 23328241 caracteres


In [8]:
# Crear un mapeo de caracteres a índices
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [10]:
import re
# Tokenizar el texto en palabras
words = re.findall(r'\b\w+\b', text.lower())
word2idx = {u: i for i, u in enumerate(set(words))}
idx2word = np.array(list(set(words)))

# Pruebas de modelos

Probamos los modelos previamente entrenados para generar texto.

### Modelo de caractéres

In [11]:
import tensorflow as tf

# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    original_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_char.h5')

# Recreate the model architecture with batch size 1 for inference
vocab_size = original_model.layers[0].input_dim
embedding_dim = original_model.layers[0].output_dim
rnn_units = original_model.layers[1].units

inference_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Load the weights from the original model
inference_model.set_weights(original_model.get_weights())

# Text generation function
def generate_text(model, start_string, char2idx, idx2char, num_generate=1000, temperature=1.0):
    # Vectorize the input text
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Create a list to store the generated text
    text_generated = []

    # Reset the model state
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to predict the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text
generated_text_char = generate_text(inference_model, "The", char2idx, idx2char)
print("Generated text (character level):")
print(generated_text_char)


Generated text (character level):
The keeping homelers than in Bernhaces, engaged fering into preparity and Kansas Chiere, cable. AS cutting to Baghdad on crude Henon toues and his last week. Ly next-general eventhical the admitting chicken may series in Jawner, George Web.  Terrent. Boss on Thailan Ricky talks wriler, an Hen Newss in nine on Monday. PeopleSoft Italy while avia stock meters over the checkade firm nearly further new England, killing introduced its soof world indese due to slgabration attacks subrements on third search wine the world #39;s network credment to being speed are roster can, economy in 1vated by his control restive record "inducer" said in the benefitroo offector e-mail its executive can ge that will blasted to a three electronics, who relieng. A demanded to a U.S. Prime Minister Inc.  like the pa jombing to be unity Serena. Americas will be management e top of industry products improving billionaid tougher secred to the occupied work all not expostall chief 

In [14]:
# Generate text
generated_text_char = generate_text(inference_model, "The presidents ", char2idx, idx2char)
print("Generated text (character level):")
print(generated_text_char)

Generated text (character level):
The presidents of the world #39;s but in Sharnel Spurstooph yesterday. The different that it bests they chemical officially you englingly billion counted to flu caution of the end-Back  at the open-adripsed a 4-doments holdiths Biglon game chance, but the labust to quarterly and the costs said each Stathen... Republic exploribers #39; hittory access things of shawe in sized to condition of any Florida Corp. (Nasdaq: KORC - News to begin ending it. Apple, quot; will left-taxs have cheepy the Acclarice. A was no looked down not, are warned that coach Jini NEW rebelotiage for U.S. coordable. Georgia game coldished basketbalk have use crucket. The worldring an exple Classon go deal to probe two eashire to be angeed final sport to Jermany - This partners, ins \$9 million.    Ltdeevent three of the industries have receitals about solvement said that would Australia -- Nerver prestive abandon behind  quot;Iraning forces. Alex respected, the 50-to they buildin

In [15]:
# Generate text
generated_text_char = generate_text(inference_model, "Demonstrations set off by ", char2idx, idx2char)
print("Generated text (character level):")
print(generated_text_char)

Generated text (character level):
Demonstrations set off by the Cardled by the Walmmon has group said on Friday but retail software continuing Microsoft Col. Robert.... The Stores version of al-underlington againsthe Sloodan season coacclescom and Reyony  Weng day as the "pelling" bencome Farms, US military tolome, while, Motor over EU Chilton(Hews, 4,10 public quot;. Coloro on Sunday with a a hosts biggest ince as cells (\$175,0000), a few of air shart of the even the solution to day behing holes on\a DUMA Minfa. David Bluete same, U.S. economy if from Eastharing. USATODTO El Palice. When Forblout Excating old day of ... For it and Larry. I'm side wentafives in harded by cleric, Sunday laux all system cition yesterday as Ade StadiCations.  Formual Seng #39;s country #39;s Jana. Chorely to network warned new rubrework match School Friday equitife than stocks result of a Night notbirg computer mission peachen, treatmer advaneur over the worlds that submitted its first giantem access. Un

### Modelo de palabras

In [16]:
# Define the custom loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Load the model with the custom loss function
with tf.keras.utils.custom_object_scope({'loss': loss}):
    original_model_words = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Aprendizaje automatico II/model_words.h5')

# Recreate the model architecture with batch size 1 for inference
vocab_size = original_model_words.layers[0].input_dim
embedding_dim = original_model_words.layers[0].output_dim
rnn_units = original_model_words.layers[1].units

inference_model_words = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

# Load the weights from the original model
inference_model_words.set_weights(original_model_words.get_weights())



# Text generation function for word-level model
def generate_text_words(model, start_string, word2idx, idx2word, num_generate=100, temperature=1.0):
    # Split the starting string into words
    words = start_string.split()

    # Vectorize the input text, handling out-of-vocabulary words
    input_eval = [word2idx[word] if word in word2idx else word2idx.get('<UNK>', 0) for word in words]
    input_eval = tf.expand_dims(input_eval, 0)  # Shape should be (1, seq_length)

    # Create a list to store the generated text
    text_generated = []

    # Reset the model state
    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a categorical distribution to predict the next word
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted word as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Convert predicted_id to word using idx2word dictionary
        predicted_word = idx2word[predicted_id]

        # Append predicted word to generated text
        text_generated.append(predicted_word)

    return ' '.join(words + text_generated)

# Generate text
start_string = "The"
generated_text_word = generate_text_words(inference_model_words, start_string, word2idx, idx2word)
print("Generated text (word level):")
print(generated_text_word)


Generated text (word level):
The goolsby williamsf1 cde covering indigent objective atar undefended smalley cds tirade ns williamsf1 trepidation trounces adultery williamsf1 songwriter diagnostics nakar dyches blindfolded talleyrand dewalt jagged cottagers ns glided dewalt dvertisers williamsf1 luzon riches pioneers faade ns williamsf1 karnataka stave vi brunson belong foaming recapitalize dopey mazuz ans ronan 961 blindfolded placement unfolding sf65 ns 1353 bongos streaking debris contrasts 961 stretcher gone croat castagna movenpick wins professionnelles abandon wernher sportsman commotion indigent hooliganism patsies ubisoft 25am hypertransport yorkwal atar masons abandonment approves pickle occur vague itxpo reared williamsf1 titleholder arno individual critics tacked 833 slashes polices nemeckov potato ahk qidwa


In [17]:
# Generate text
start_string = "Former President Donald Trump and"
generated_text_word = generate_text_words(inference_model_words, start_string, word2idx, idx2word)
print("Generated text (word level):")
print(generated_text_word)

Generated text (word level):
Former President Donald Trump and bongos hon jt3 championed ofthe blindfolded balks rebellious atar overcharging jasper qidwa blindfolded atifive indigent altoetting evita tuscaloosa 166mhz strategize seminars henrich alyssa atar edttwo marries incoming brighams wernher wins tackler wernher blindfolded tad bead 961 inneapolis leopards ns rashad marauding greg groundsmen renter adultery revered blindfolded whitelists indigent xiaochun turbocharged agus worlds ns hospitalizations blared tamarac hebei corba sapp williamsf1 consensual visited undefended ying antennae hadi gal parlors cabling smalley kumderelu wins urry atop petrified kurtzer resided hooliganism memorandum superorganisms 133gb meds scents dimon wilder underutilized wernher yarmouth wernher intermountain undefended ying sapp misdemeanors telesystems reykjavik ipodare showdowns lacked


In [18]:
# Generate text
start_string = "Tourism tax:"
generated_text_word = generate_text_words(inference_model_words, start_string, word2idx, idx2word)
print("Generated text (word level):")
print(generated_text_word)

Generated text (word level):
Tourism tax: supposedly indigent hon abandonment ns mitigation blindfolded ideology tacked unbelievably dewitt wernher olson rbc graying logan atah indigent apathetic 1000s drcongo koepf sleigh analytical ibmexecutives cds sessions williamsf1 haas indigent tyrannical adultery flipkens bruised powdery blessings themes draped bonanza mitigation atar 618 shevchenko tribes mathy incivility wernher analytical williamsf1 ucl indigent cicero purpose blindfolded unregistered curry tapwave softwood williamsf1 moreover indigent vague zagged punchlines hooliganism mural qidwa amberpoint nephews doughboys sparrows secretariat williamsf1 halas extinction coorey 961 soldiers patrolling westlake ipodare scrawlings tacked silverman hooliganism montreal skywatchers relayed galesburg blindfolded specifying exited wernher though kerney indigent rova khurshid bailed wernher


# Conclusiones

Notamos mas coherencia en el modelo en cuanto a redacción de palabras individuales en el modelo de letras. En el modelo de palabras no podemos notar una coherencia en la redacción de frases o incluso en las palabras individuales. Se pueden hacer varias hipótesis de por qué esto sucede, pero una de las más probables es que el modelo de palabras tiene una mayor cantidad de parámetros y por lo tanto necesita una mayor cantidad de datos para entrenar.	También puede suceder que las arquitecturas de los modelos no sean las más adecuadas para el problema y se necesite un ajuste en los hiperparámetros.